## 🔹 Introduction to ANN Algorithms  

**Approximate Nearest Neighbors (ANN)** algorithms are designed to find vectors in a **high-dimensional space** that are close to a query vector, based on a distance metric (e.g., **Euclidean**, **Cosine**).  

Unlike exact nearest neighbor search, ANN sacrifices some **accuracy** for significant **speed improvements**, making it ideal for **large-scale applications** like **RAG**, where embeddings (e.g., from BERT or other LLMs) need to be searched quickly.  

In **RAG**, ANN is used to retrieve relevant documents or embeddings from a **vector database** to **augment the generation process**.  

### ✅ Popular ANN Algorithms  

- **HNSW (Hierarchical Navigable Small World)**  
  - Graph-based method.  
  - Builds a **multi-layered structure** for fast and accurate searches.  

- **IVF (Inverted File Index)**  
  - Quantization-based method.  
  - **Clusters vectors** and searches within the closest clusters.  

- **LSH (Locality-Sensitive Hashing)**  
  - Hashing-based method.  
  - Maps **similar vectors** to the **same hash buckets** for fast retrieval.  


## 🔹 HNSW (Hierarchical Navigable Small World)  

### 3.1 Theory  

**HNSW** is a **graph-based ANN algorithm** that builds a hierarchical structure of interconnected nodes (vectors) to enable **fast searches**.  

It is based on the **Navigable Small World (NSW)** graph, where each node is connected to a **small set of neighbors**, ensuring short paths between any two nodes.  

---

### 📐 Structure  
- Organizes vectors into **multiple layers**.  
- Higher layers → **sparse** (fewer nodes).  
- Lower layers → **denser**.  
- The **top layer** contains only a few nodes, while the **bottom layer** contains *all vectors*.  

---

### 🔎 Search Process  
1. **Start at the top layer** and find the closest node to the query.  
2. **Move to the next layer**, using the previous layer’s result as the entry point.  
3. **Repeat until the bottom layer**, refining the neighbor list at each step.  


In [4]:
import numpy as np
import hnswlib
from sklearn.metrics.pairwise import euclidean_distances
import time

# -------------------------
# Step 1: Generate synthetic embeddings
# -------------------------
np.random.seed(42)
dimension = 128          # Embedding dimension
num_elements = 50000     # Number of "documents"
data = np.random.random((num_elements, dimension)).astype('float32')

# Create a query vector
query = np.random.random((1, dimension)).astype('float32')

# -------------------------
# Step 2: Brute force search (for comparison)
# -------------------------
start = time.time()
distances = euclidean_distances(query, data)[0]
top_k = np.argsort(distances)[:5]
brute_force_time = time.time() - start

print("🔎 Brute Force Search")
print(f"Top 5 neighbors: {top_k}")
print(f"Distances: {distances[top_k]}")
print(f"Time taken: {brute_force_time:.4f} seconds\n")

# -------------------------
# Step 3: HNSW Indexing
# -------------------------
index = hnswlib.Index(space='l2', dim=dimension)  # 'l2' = Euclidean distance
index.init_index(max_elements=num_elements, ef_construction=200, M=16)

# Add items to index
index.add_items(data, np.arange(num_elements))

# ef controls accuracy/speed trade-off
index.set_ef(50)

# -------------------------
# Step 4: HNSW Search
# -------------------------
start = time.time()
labels, distances = index.knn_query(query, k=5)
hnsw_time = time.time() - start

print("⚡ HNSW Search")
print(f"Top 5 neighbors: {labels[0]}")
print(f"Distances: {distances[0]}")
print(f"Time taken: {hnsw_time:.6f} seconds")


🔎 Brute Force Search
Top 5 neighbors: [ 7461 30543 35161 18054 33682]
Distances: [3.6132355 3.6199749 3.6956348 3.7309732 3.7433467]
Time taken: 0.0631 seconds

⚡ HNSW Search
Top 5 neighbors: [35161 15105 38889  8386  3244]
Distances: [13.657718 14.171591 14.38143  14.567127 14.903164]
Time taken: 0.000000 seconds


## 🔹 IVF (Inverted File Index)  

### 4.1 Theory  

**IVF** is a **quantization-based ANN algorithm** that partitions the vector space into **clusters** using a clustering algorithm (e.g., **k-means**).  
Each vector is assigned to the **nearest cluster centroid**, and search is performed within a **subset of clusters** instead of the entire dataset.  

---

### 📐 Structure  
- **Codebook**: A set of cluster centroids (learned via k-means).  
- **Inverted Lists**: Each centroid points to a list of vectors assigned to that cluster.  

---

### 🔎 Search Process  
1. **Find the `nprobe` closest centroids** to the query vector.  
2. **Search for nearest neighbors** within the inverted lists of those centroids.  


In [6]:
import numpy as np
import faiss

# Generate synthetic data
np.random.seed(42)
dimension = 128
num_elements = 10000
data = np.random.random((num_elements, dimension)).astype('float32')
query = np.random.random((1, dimension)).astype('float32')

# Train the IVF index
nlist = 100  # Number of clusters
quantizer = faiss.IndexFlatL2(dimension)  # Flat index for quantizer
index = faiss.IndexIVFFlat(quantizer, dimension, nlist, faiss.METRIC_L2)
index.train(data)  # Train the clustering model
index.add(data)    # Add vectors to the index

# Set number of clusters to probe during search
index.nprobe = 10

# Query the index for top-5 nearest neighbors
k = 5
distances, labels = index.search(query, k)

print(f"Top {k} nearest neighbors (indices): {labels[0]}")
print(f"Distances: {distances[0]}")

Top 5 nearest neighbors (indices): [8769 9571 4436 1056  556]
Distances: [13.34696   14.837141  15.379183  15.488777  15.4977865]


## 🔹 LSH (Locality-Sensitive Hashing)  

### 5.1 Theory  

**LSH** is a **hashing-based ANN algorithm** that maps similar vectors to the **same hash buckets** with high probability.  
It uses **hash functions designed to preserve locality**, meaning similar vectors produce similar hashes.  

---

### 📐 Structure  
- **Hash Functions**: Random projections or other locality-sensitive functions that map vectors to buckets.  
- **Hash Tables**: Multiple hash tables store vectors, increasing the chance of finding neighbors.  

---

### 🔎 Search Process  
1. **Hash the query vector** to identify candidate buckets.  
2. **Search for nearest neighbors** within those buckets.  


In [ ]:
import numpy as np
import faiss

# Generate synthetic data
np.random.seed(42)
dimension = 128
num_elements = 10000
data = np.random.random((num_elements, dimension)).astype('float32')
query = np.random.random((1, dimension)).astype('float32')

# Initialize LSH index
nbits = dimension * 4  # Number of bits for hashing
index = faiss.IndexLSH(dimension, nbits)

# Add vectors to the index
index.add(data)

# Query the index for top-5 nearest neighbors
k = 5
distances, labels = index.search(query, k)

print(f"Top {k} nearest neighbors (indices): {labels[0]}")
print(f"Distances: {distances[0]}")

Top 5 nearest neighbors (indices): [2980 4436 5238 7666 1032]
Distances: [92. 92. 92. 92. 94.]


: 

## 🔹 Comparing HNSW, IVF, and LSH  

| Algorithm | Type              | Accuracy | Speed      | Memory | Best Use Case in RAG                          |
|-----------|-------------------|----------|------------|--------|-----------------------------------------------|
| **HNSW**  | Graph-based       | High     | Fast       | High   | High-accuracy retrieval, moderate datasets    |
| **IVF**   | Quantization-based| Medium   | Fast       | Medium | Large datasets, balanced speed/accuracy       |
| **LSH**   | Hashing-based     | Low      | Very Fast  | Low    | Very large datasets, coarse retrieval         |


In [ ]:
import numpy as np
import hnswlib
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM

# Step 1: Generate document and query embeddings
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

# Sample documents
documents = [
    "The capital of France is Paris.",
    "Germany is known for its engineering and automotive industries.",
    "Japan has a rich cultural heritage, including tea ceremonies and samurai traditions."
]
doc_embeddings = np.vstack([get_embedding(doc) for doc in documents])

# Step 2: Build HNSW index
dimension = doc_embeddings.shape[1]
index = hnswlib.Index(space='cosine', dim=dimension)
index.init_index(max_elements=len(documents), ef_construction=200, M=16)
index.add_items(doc_embeddings, np.arange(len(documents)))
index.set_ef(50)

# Step 3: Query the index
query = "What is the capital of France?"
query_embedding = get_embedding(query)
k = 1
labels, distances = index.knn_query(query_embedding, k=k)

# Step 4: Retrieve documents and generate response
retrieved_doc = documents[labels[0][0]]
generator = AutoModelForCausalLM.from_pretrained("gpt2")
gen_tokenizer = AutoTokenizer.from_pretrained("gpt2")
prompt = f"Context: {retrieved_doc}\nQuestion: {query}\nAnswer:"
inputs = gen_tokenizer(prompt, return_tensors="pt")
outputs = generator.generate(**inputs, max_length=50)
answer = gen_tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"Retrieved Document: {retrieved_doc}")
print(f"Generated Answer: {answer}")